In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Read the text file
with open('sherlock-holm.es_stories_plain-text_advss.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [3]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [4]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [5]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [6]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [7]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [12]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=25, verbose=1)

Epoch 1/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 50s 51ms/step - accuracy: 0.4407 - loss: 2.6167
Epoch 2/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 47s 50ms/step - accuracy: 0.4974 - loss: 2.3927
Epoch 3/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 80s 48ms/step - accuracy: 0.5506 - loss: 2.1348
Epoch 4/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.6001 - loss: 1.8928
Epoch 5/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 82s 49ms/step - accuracy: 0.6425 - loss: 1.6964
Epoch 6/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 82s 50ms/step - accuracy: 0.6782 - loss: 1.5334
Epoch 7/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 45s 48ms/step - accuracy: 0.7061 - loss: 1.3975
Epoch 8/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 83s 49ms/step - accuracy: 0.7458 - loss: 1.2387
Epoch 9/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 83s 50ms/step - accuracy: 0.7647 - loss: 1.1385
Epoch 10/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 80s 47ms/step - accuracy: 0.7962 - loss: 1.0222
Epoch 11/25
942/942 ━━━━━━━━━━━━━━━━━━━━ 45s 48ms/step - accuracy: 0.8152 - loss: 0.9247
Epoch 12/25
942/942 ━━━━━━━━━━

In [19]:
seed_text = "i will visit that place if"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
i will visit that place if you care to
